In [82]:
# import google earth engine
import ee

# authenticate to the Earth Engine servers
ee.Authenticate()

# initialize the Earth Engine object, using the authentication credentials
ee.Initialize()


In [83]:
# load s2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')

# create bounding box
bbox = ee.Geometry.Rectangle([98.9853, 18.7883, 98.9853, 18.7883])

# filter s2 image collection
s2 = s2.filterDate('2020-01-01', '2024-12-31').filterBounds(bbox)

# get s2 image collection size
print(s2.size().getInfo())

# get s2 image collection first image
img = s2.first()

# show s2 image collection first image band names
print(img.bandNames().getInfo())

# show s2 image collection first image band scale
print(img.select('B2').projection().nominalScale().getInfo())

# show band statistics
print(img.select('B2').reduceRegion(reducer=ee.Reducer.minMax(), geometry=bbox, scale=10).getInfo())


365
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']
10
{'B2_max': None, 'B2_min': None}


In [116]:
# import folium and display the image
import folium   

map = folium.Figure(height="300px")
m = folium.Map(location=[18.7883, 98.9853],
               zoom_start=10).add_to(map)

# create function add_ee_layer

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# add_ee_layer function to folium
folium.Map.add_ee_layer = add_ee_layer

# create NDVI function
def ndvi(img):
    return img.normalizedDifference(['B8', 'B4']).rename('nd')

# calculate NDVI and add band to image
img = img.addBands(ndvi(img))

# get s2 properties
properties = img.propertyNames().getInfo()
# print(properties)

# for i in properties:
#     print(i + ':', img.get(i).getInfo())

# add color palette for NDVI
palette = ['ff0000', 'ffff00', '00c800', '006400']

# add NDVI to folium map
m.add_ee_layer(img.select('nd'), {'palette': palette, 'min': -0.2, 'max': 0.8 }, 'NDVI')

m.add_ee_layer(img, {'bands': ['nd'], 'min': -1, 'max': 1}, 'NDVI2')

# add s2 image to folium map
m.add_ee_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 500, 'max': 2000}, 'S2')

# add control layer to folium map
folium.LayerControl().add_to(m)

# show folium map
m